In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-02,France,131141.0,0.0,24892.0,0.0,0.0,0.0
0,2020-05-02,China,84051.0,0.0,4666.0,0.0,0.0,0.0
0,2020-05-02,Italy,209269.0,0.0,28506.0,0.0,0.0,0.0
0,2020-05-02,Spain,215371.0,0.0,25059.0,0.0,0.0,0.0
0,2020-05-02,United States,1135545.0,0.0,67208.0,0.0,0.0,0.0
0,2020-05-02,World,3396558.0,0.0,244256.0,0.0,0.0,0.0
0,2020-05-02,United Kingdom,183756.0,0.0,28208.0,0.0,0.0,0.0
0,2020-05-02,Germany,161928.0,0.0,6684.0,0.0,0.0,0.0
0,2020-05-02,Iran,96609.0,0.0,6169.0,0.0,0.0,0.0
0,2020-05-02,Turkey,124743.0,0.0,3343.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-03,France,131141.0,130979.0,24892.0,24760.0,162.0,132.0
0,2020-05-03,China,84051.0,83961.0,4666.0,4637.0,90.0,29.0
0,2020-05-03,Italy,209269.0,209328.0,28506.0,28710.0,-59.0,-204.0
0,2020-05-03,Spain,215371.0,216582.0,25059.0,25100.0,-1211.0,-41.0
0,2020-05-03,United States,1135545.0,1133069.0,67208.0,66385.0,2476.0,823.0
0,2020-05-03,World,3396558.0,3388665.0,244256.0,243312.0,7893.0,944.0
0,2020-05-03,United Kingdom,183756.0,182260.0,28208.0,28131.0,1496.0,77.0
0,2020-05-03,Germany,161928.0,162496.0,6684.0,6649.0,-568.0,35.0
0,2020-05-03,Iran,96609.0,96448.0,6169.0,6156.0,161.0,13.0
0,2020-05-03,Turkey,124743.0,124375.0,3343.0,3336.0,368.0,7.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")